In [13]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

'C:\\Users\\daniel.martinezcarre\\.wdm\\drivers\\geckodriver\\win64\\v0.34.0\\geckodriver.exe'

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By

### Variables

In [15]:
URL="https://contrataciondelestado.es/"

In [16]:
driver = webdriver.Firefox()

### Carga de la URL

In [17]:
driver.get(URL)

### Acceso a expedientes

> Publicaciones

In [21]:
link = driver.find_element(By.XPATH, "//a[@href='/wps/portal/licitaciones']")
link.click()

> Licitaciones

In [22]:
link_licitaciones = driver.find_element(By.CLASS_NAME, "divLogo")
link_licitaciones.click()

> Búsqueda avanzada

In [23]:
link_busqueda_avanzada = driver.find_element(By.ID, "viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:textBusquedaAvanzada")
link_busqueda_avanzada.click()

> Organización contratante → Seleccionar

In [24]:
link_busqueda_avanzada = driver.find_element(By.ID, "viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:idSeleccionarOCLink")
link_busqueda_avanzada.click()

> Entidades locales → Galicia → Coruña → Ayuntamientos → A Coruña

In [77]:
sds = driver.find_element(By.ID, "tafelTree_maceoArbol_id_3")
sds.click()

In [72]:
main_div = driver.find_element(By.CSS_SELECTOR, "div.tafelTree_root")

div_rows = main_div.find_elements(By.CSS_SELECTOR, "div.tafelTree_row")

print(len(div_rows))
for i in div_rows:
    print(f"{i}\n")

# for i, div_row in enumerate(div_rows):
    # if i == 2:
        # img_element = div_row.find_element(By.CSS_SELECTOR, "img.tafelTreeopenable")
# 
        # img_element.click()
        # break

89
<selenium.webdriver.remote.webelement.WebElement (session="90e0da32-916c-4786-8117-c3de582d2881", element="19ffe178-e21b-4b7e-a955-e0256878845b")>

<selenium.webdriver.remote.webelement.WebElement (session="90e0da32-916c-4786-8117-c3de582d2881", element="b43888dd-c4ce-4abd-a373-194ac56043bd")>

<selenium.webdriver.remote.webelement.WebElement (session="90e0da32-916c-4786-8117-c3de582d2881", element="d6217c74-1537-4483-9756-a12f0ad003db")>

<selenium.webdriver.remote.webelement.WebElement (session="90e0da32-916c-4786-8117-c3de582d2881", element="26322e48-3a1c-44e6-a4f7-4df7c6cff999")>

<selenium.webdriver.remote.webelement.WebElement (session="90e0da32-916c-4786-8117-c3de582d2881", element="0d0c8017-b860-4263-9a1f-057f06d2b128")>

<selenium.webdriver.remote.webelement.WebElement (session="90e0da32-916c-4786-8117-c3de582d2881", element="2447c97d-771f-4fe7-8aca-909bd602cc97")>

<selenium.webdriver.remote.webelement.WebElement (session="90e0da32-916c-4786-8117-c3de582d2881", element="e3